In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
# Accessing data
import os
cwd = '/content/drive/MyDrive/Graduate_Thesis/colab/data_files-20210512T051256Z-001/data_files'
print(cwd)

In [ ]:
filename = ['200421_TM_RDE.csv','200422_TM_RDE.csv','200423_TM_RDE.csv','200424_TM_RDE.csv']

training_data_file = filename
print(training_data_file)

In [ ]:
output_variable_names = [
    'CAL_NOxFlowrateLNTOutletValid_gps',
    'AUX_O2FracLNTOutlet_volPerc',
    'AUX_SensTempLNTOutlet_degC',
]

variable_names_to_extract = [ 
    'AUX_NOxLNInlet_ppm', 
#     'AUX_NOxLNTOutlet_ppm',
#    'CAL_Time_sec', 
#    'AUX_RTCDate_YYYYMMDD', 
   'OBD_CalEngLoad_perc',
    'OBD_EngCoolantTemp_degC', 
    'OBD_EngineSpeed_rpm', 
#    'OBD_VehicleSpeed_kmph', 
#    'OBD_ActEngPerTorque_perc',
#    'OBD_EngRefTorque_Nm', 
#    'OBD_MAFSensor_gps', 
#    'OBD_EGT11_degC', 
#    'OBD_EGT12_degC',
#    'OBD_DPFDiffPress_kPa', 
#    'AUX_NOxSCROutlet_ppm', 
#    'AUX_lambdaLNTInlet_none', 
#    'AUX_lambdaLNTOutlet_none',
#    'AUX_lambdaSCROutlet_none', 
    'AUX_O2FracLNTInlet_volPerc', 
    'AUX_O2FracLNTOutlet_volPerc',
#    'AUX_O2FracSCROutlet_volPerc',
#    'AUX_SensAmbTemp_degC',
#    'AUX_SensAmbRH_perc',
#    'AUX_SensBaroPress_kPa',
    'AUX_SensTempTurbinOutlet_degC',
#    'AUX_SensTempInManiInlet_degC',
#    'AUX_SensTempTurbinInlet_degC',
    'AUX_SensTempLNTOutlet_degC',
#    'AUX_SensTempSCROutlet_degC',
#    'AUX_SensTempCompressorOutlet_degC',
#    'AUX_SensPressEGRCoolerInlet_absBar',
#    'AUX_SensDiffPressDPF_kPa',
#    'AUX_SensDiffPressSCR_kPa',
    'CAL_ExhFlowrate_gps',
#    'CAL_CO2Flowrate_gphr',
   'CAL_NOxFlowrateLNTInletValid_gps',
   'CAL_NOxFlowrateLNTOutletValid_gps',
#    'CAL_NOxFlowrateSCROutletValid_gps'
    ]
print(len(variable_names_to_extract))

In [ ]:
full_data = []
i = 0

for file_name in training_data_file:
    file_path = os.path.join(cwd,file_name)
    load_data = pd.read_csv(file_path, usecols=variable_names_to_extract)
    if i == 0:
        full_data = load_data
    else:
        full_data = full_data.append(load_data, ignore_index=True, sort=False)
    i += 1

In [ ]:
full_data.shape

In [ ]:
full_data.head()

In [ ]:
full_data.info()

In [ ]:
full_data.describe()

In [ ]:
# Cleaning data
RPM_min = 100                      # excluding engine-stop
NOx_max = 1649                     # excluding clipped NOx data (since the sensor maxed out at 1650 ppm)

full_data = full_data.loc[((full_data['OBD_EngineSpeed_rpm'] >= RPM_min) & (full_data['AUX_NOxLNInlet_ppm'] <= NOx_max))]
full_data = full_data.drop(['AUX_NOxLNInlet_ppm'], axis=1)

In [ ]:
full_data.shape

In [ ]:
cleaned_up_data = full_data.dropna()                      # eliminating rows with NaN in any column(s)
print(cleaned_up_data.shape)
print(cleaned_up_data.isnull().values.any())

In [ ]:
# Dividing data in 5 equally
cleaned_up_data['LNT_inlet_NOx_cat'] = pd.qcut(cleaned_up_data['CAL_NOxFlowrateLNTInletValid_gps'],
                                               q=5,
                                               labels=[1, 2, 3, 4, 5])
cleaned_up_data['LNT_inlet_NOx_cat'].hist()
cleaned_up_data['LNT_inlet_NOx_cat'].value_counts()/len(cleaned_up_data)

In [ ]:
cleaned_up_data.info()

In [ ]:
cleaned_up_data.head()

In [ ]:
print(len(cleaned_up_data))
reindexed_data = cleaned_up_data.reset_index(drop=True)
print(len(reindexed_data))
reindexed_data['LNT_inlet_NOx_cat'].value_counts()/len(reindexed_data)

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_validation_index, test_index in split.split(reindexed_data, reindexed_data['LNT_inlet_NOx_cat']):
    train_validation_set = reindexed_data.loc[reindexed_data.index.intersection(train_validation_index),:]
    test_set = reindexed_data.loc[reindexed_data.index.intersection(test_index),:]

In [ ]:
len(train_validation_set)

In [ ]:
len(test_set)

In [ ]:
reindexed_train_validation_set = train_validation_set.reset_index(drop=True)
print(len(reindexed_train_validation_set))

In [ ]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, validation_index in split.split(reindexed_train_validation_set, reindexed_train_validation_set['LNT_inlet_NOx_cat']):
    train_set = reindexed_train_validation_set.loc[reindexed_train_validation_set.index.intersection(train_index),:]
    validation_set = reindexed_train_validation_set.loc[reindexed_train_validation_set.index.intersection(validation_index),:]

In [ ]:
X_train = train_set
y_train = pd.concat([X_train.pop(x) for x in output_variable_names], axis=1)

X_valid = validation_set
y_valid = pd.concat([X_valid.pop(x) for x in output_variable_names], axis=1)

X_test = test_set
y_test = pd.concat([X_test.pop(x) for x in output_variable_names], axis=1)

In [ ]:
for set_ in (train_set, validation_set, test_set):
    set_.drop("LNT_inlet_NOx_cat", axis=1, inplace=True)

In [ ]:
print(X_train.keys())

In [ ]:
print(y_train.keys())

In [ ]:
# from sklearn.tree import DecisionTreeRegressor
# tree_reg = DecisionTreeRegressor()
# tree_reg.fit(X_train, y_train)

In [ ]:
# from sklearn.model_selection import cross_val_score
# scores = cross_val_score(tree_reg, X_train, y_train,
#                          scoring="neg_mean_squared_error", cv=5)
# tree_rmse_scores = np.sqrt(-scores)

In [ ]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[X_train.shape[1]]):
    model = keras.models.Sequential()
    options = {"input_shape": input_shape}
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu", **options))
        options = {}
    model.add(keras.layers.Dense(1, **options))
    optimizer = tf.keras.optimizers.SGD(learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [ ]:
keras_reg = tf.keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [ ]:
mean_vec = X_train.mean()
std_vec = X_train.std()
print(mean_vec)
print(std_vec)

mean_NOx_gps = mean_vec['CAL_NOxFlowrateLNTInletValid_gps']
std_NOx_gps = std_vec['CAL_NOxFlowrateLNTInletValid_gps']

mean_O2_frac = mean_vec['AUX_O2FracLNTInlet_volPerc']
std_O2_frac = std_vec['AUX_O2FracLNTInlet_volPerc']

mean_exhT_degC = mean_vec['AUX_SensTempTurbinOutlet_degC']
std_exhT_degC = std_vec['AUX_SensTempTurbinOutlet_degC']


#use json
mean_data = [dict(mean_vec)]
std_data = [dict(std_vec)]

import json
with open('mean_data_w_20_RDEs_only.json', 'w') as f:
    json.dump(mean_data,f)
    
with open('std_data_w_20_RDEs_only.json', 'w') as f:
    json.dump(std_data,f)

In [ ]:
# normalization
X_train = (X_train - mean_vec) / std_vec
y_train['CAL_NOxFlowrateLNTOutletValid_gps'] = (y_train['CAL_NOxFlowrateLNTOutletValid_gps'] - mean_NOx_gps) / std_NOx_gps
y_train['AUX_O2FracLNTOutlet_volPerc'] = (y_train['AUX_O2FracLNTOutlet_volPerc'] - mean_O2_frac) / std_O2_frac
y_train['AUX_SensTempLNTOutlet_degC'] = (y_train['AUX_SensTempLNTOutlet_degC'] - mean_exhT_degC) / std_exhT_degC

X_valid = (X_valid - mean_vec) / std_vec
y_valid['CAL_NOxFlowrateLNTOutletValid_gps'] = (y_valid['CAL_NOxFlowrateLNTOutletValid_gps'] - mean_NOx_gps) / std_NOx_gps
y_valid['AUX_O2FracLNTOutlet_volPerc'] = (y_valid['AUX_O2FracLNTOutlet_volPerc'] - mean_O2_frac) / std_O2_frac
y_valid['AUX_SensTempLNTOutlet_degC'] = (y_valid['AUX_SensTempLNTOutlet_degC'] - mean_exhT_degC) / std_exhT_degC

X_test = (X_test - mean_vec) / std_vec
y_test['CAL_NOxFlowrateLNTOutletValid_gps'] = (y_test['CAL_NOxFlowrateLNTOutletValid_gps'] - mean_NOx_gps) / std_NOx_gps
y_test['AUX_O2FracLNTOutlet_volPerc'] = (y_test['AUX_O2FracLNTOutlet_volPerc'] - mean_O2_frac) / std_O2_frac
y_test['AUX_SensTempLNTOutlet_degC'] = (y_test['AUX_SensTempLNTOutlet_degC'] - mean_exhT_degC) / std_exhT_degC

In [ ]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_valid = np.array(X_valid)
y_valid = np.array(y_valid)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
import time

start = time.time()

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import keras
param_distribs = {
    "n_hidden": [2, 3, 4, 5, 6],
    "n_neurons": [32, 64, 128, 256, 512], ## 32~512 32 간격/2^n
    "learning_rate": [5e-3, 5e-4, 5e-5], ## 중앙: 5e-4, 5e-3, 5e-5 / initilal, decay
}
rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=5)
rnd_search_cv.fit(X_train, y_train, epochs=50,
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

In [ ]:
print("계산시간 :", time.time() - start)

In [ ]:
rnd_search_cv.best_params_

In [ ]:
-rnd_search_cv.best_score_

In [ ]:
 model = rnd_search_cv.best_estimator_.model

 model.compile(optimizer='rmsprop',
             loss='mse',
             metrics=['mse'])

model.summary()

In [ ]:
import time

start = time.time()

In [ ]:
history = model.fit(X_train, y_train, epochs=400, batch_size=128, validation_data=(X_valid, y_valid))

In [ ]:
print("계산시간 :", time.time() - start)

In [ ]:
loss = history.history['loss']
mse = history.history['mse']
val_loss = history.history['val_loss']
val_mse = history.history['val_mse']

In [ ]:
import matplotlib.pyplot as plt

epochs = range(len(loss))

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

plt.plot(epochs, mse, 'bo', label='Training MSE')
plt.plot(epochs, val_mse, 'r', label='Validation MSE')
plt.title('Training and validation MSE')
plt.legend()
plt.show()

In [ ]:
y_pred = model.predict(X_train)

In [ ]:
plt.plot(y_pred, y_train, 'o')
x = np.arange(-2,7)
y = x
plt.plot(x, y, 'k')
plt.show()

In [ ]:
from sklearn.metrics import r2_score
print(r2_score(y_pred, y_train))

In [ ]:
y_pred_test = model.predict(X_test)

In [ ]:
plt.plot(y_pred_test, y_test, 'o')
x = np.arange(-2,7)
y = x
plt.plot(x, y, 'k')
plt.show()

In [ ]:
print(r2_score(y_pred_test, y_test)) 

In [ ]:
model.save('LNT2LNT')

In [ ]:
new_model = tf.keras.models.load_model('LNT2LNT')

# Check its architecture
new_model.summary()